In [24]:
import pandas as pd 
import numpy as np
from scipy.sparse import hstack
import scipy as sp
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,chi2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [50]:
path= '/content/gdrive/MyDrive/Data-Set_Assig_1/Data-Set_Assig_1/FNC-1/train_bodies.csv'
train_bodies=pd.read_csv(path)
path= '/content/gdrive/MyDrive/Data-Set_Assig_1/Data-Set_Assig_1/FNC-1/competition_test_bodies.csv'
test_bodies=pd.read_csv(path)
path= '/content/gdrive/MyDrive/Data-Set_Assig_1/Data-Set_Assig_1/FNC-1/train_stances.csv'
train_stances=pd.read_csv(path)
path= '/content/gdrive/MyDrive/Data-Set_Assig_1/Data-Set_Assig_1/FNC-1/competition_test_stances.csv'
test_stances=pd.read_csv(path)

In [51]:
fnc_train_data=pd.merge(train_bodies,train_stances,on=['Body ID'])
fnc_test_data = pd.merge(test_bodies,test_stances, on=['Body ID'])

In [7]:
fnc_total=pd.concat((fnc_train_data,fnc_test_data))

In [9]:
stance=fnc_total.Stance
fnc_total=fnc_total.drop(labels='Stance',axis=1,inplace=False)


In [11]:
tf_idf=TfidfVectorizer(min_df=5,ngram_range=(1,2))
art_tfidf_Vect=tf_idf.fit_transform(fnc_total['articleBody'].values)
Hl_tfidf_Vect=tf_idf.fit_transform(fnc_total['Headline'].values)


In [12]:
df_data = hstack((art_tfidf_Vect,Hl_tfidf_Vect)).tocsr()


In [14]:
X_train, X_test, y_train, y_test = train_test_split(df_data, stance, test_size=0.20, random_state=1)

Feature Selection

In [15]:
X_train_best=SelectKBest(chi2,k=1000).fit_transform(X_train, y_train)
X_test_best=SelectKBest(chi2,k=1000).fit_transform(X_test, y_test)


PCA

In [16]:
from sklearn.decomposition import PCA

In [18]:
X_train_best_pca=X_train_best.toarray()
X_test_best_pca=X_test_best.toarray()
pca=PCA()
start_time=time.time()
pca_train=pd.DataFrame(pca.fit_transform(X_train_best_pca))
print('Time taken to fit the PCA: ',time.time()-start_time)

Time taken to fit the PCA:  19.596946716308594


In [19]:
pca=PCA()
start_time=time.time()
pca_test=pd.DataFrame(pca.fit_transform(X_test_best_pca))
print('Time taken to fit the PCA: ',time.time()-start_time)

Time taken to fit the PCA:  5.849065780639648


Random Forest Calssifier

In [20]:
from sklearn.ensemble import RandomForestClassifier

Without Feature Selection

In [21]:
rf=RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
present_time=time.time()
rf.fit(X_train,y_train)
print(time.time()-present_time)
y_pred1 = rf.predict(X_test)


6.584036588668823


In [22]:
print(accuracy_score(y_test,y_pred1))

0.7216289712807588


In [25]:
print(classification_report(y_test, y_pred1))


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       agree       0.00      0.00      0.00      1132
    disagree       0.00      0.00      0.00       311
     discuss       0.00      0.00      0.00      2754
   unrelated       0.72      1.00      0.84     10880

    accuracy                           0.72     15077
   macro avg       0.18      0.25      0.21     15077
weighted avg       0.52      0.72      0.60     15077



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


With Feature Selection

In [26]:
rf2=RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
present_time=time.time()
rf2.fit(X_train_best,y_train)
print(time.time()-present_time)
y_pred2 = rf2.predict(X_test_best)


1.931204080581665


In [27]:
print(accuracy_score(y_test,y_pred2))

0.7216289712807588


In [28]:
print(classification_report(y_test, y_pred2))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       agree       0.00      0.00      0.00      1132
    disagree       0.00      0.00      0.00       311
     discuss       0.00      0.00      0.00      2754
   unrelated       0.72      1.00      0.84     10880

    accuracy                           0.72     15077
   macro avg       0.18      0.25      0.21     15077
weighted avg       0.52      0.72      0.60     15077



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Using PCA

In [29]:
rf_pca=RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
present_time=time.time()
rf_pca.fit(pca_train,y_train)
print(time.time()-present_time)
y_pred_pca = rf_pca.predict(pca_test)

37.91302800178528


In [30]:
print(accuracy_score(y_test,y_pred_pca))

0.7216289712807588


In [31]:
print(classification_report(y_test, y_pred_pca))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       agree       0.00      0.00      0.00      1132
    disagree       0.00      0.00      0.00       311
     discuss       0.00      0.00      0.00      2754
   unrelated       0.72      1.00      0.84     10880

    accuracy                           0.72     15077
   macro avg       0.18      0.25      0.21     15077
weighted avg       0.52      0.72      0.60     15077



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Decison Tree

In [32]:
from sklearn.tree import DecisionTreeClassifier

Without feature Selection


In [33]:
dt=DecisionTreeClassifier(random_state=0)
start_time=time.time()
dt.fit(X_train, y_train)
print(time.time()-start_time)
y_pred = dt.predict(X_test)

351.6828770637512


In [35]:
print(accuracy_score(y_test,y_pred))

0.5859919082045499


In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       agree       0.07      0.17      0.09      1132
    disagree       0.02      0.01      0.01       311
     discuss       0.25      0.09      0.13      2754
   unrelated       0.77      0.77      0.77     10880

    accuracy                           0.59     15077
   macro avg       0.28      0.26      0.25     15077
weighted avg       0.60      0.59      0.59     15077



With Feature Selection

In [34]:
dt=DecisionTreeClassifier(random_state=0)
start_time=time.time()
dt.fit(X_train_best, y_train)
print(time.time()-start_time)
y_pred = dt.predict(X_test_best)

3.271733045578003


In [37]:
print(accuracy_score(y_test,y_pred))

0.5859919082045499


In [38]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       agree       0.07      0.17      0.09      1132
    disagree       0.02      0.01      0.01       311
     discuss       0.25      0.09      0.13      2754
   unrelated       0.77      0.77      0.77     10880

    accuracy                           0.59     15077
   macro avg       0.28      0.26      0.25     15077
weighted avg       0.60      0.59      0.59     15077



Using PCA

In [39]:
dt=DecisionTreeClassifier(random_state=0)
start_time=time.time()
dt.fit(pca_train, y_train)

print(time.time()-start_time)
y_pred_pca = dt.predict(pca_test)

139.98958587646484


In [40]:
print(accuracy_score(y_test,y_pred_pca))

0.6056244610996883


In [41]:
print(classification_report(y_test, y_pred_pca))

              precision    recall  f1-score   support

       agree       0.09      0.07      0.08      1132
    disagree       0.02      0.03      0.02       311
     discuss       0.23      0.17      0.20      2754
   unrelated       0.74      0.79      0.76     10880

    accuracy                           0.61     15077
   macro avg       0.27      0.26      0.26     15077
weighted avg       0.58      0.61      0.59     15077



Naive Bayes Classifier

In [42]:
from sklearn.naive_bayes import MultinomialNB

In [43]:
Nb=MultinomialNB()
start_time=time.time()
Nb.fit(X_train, y_train)
print("time:",time.time()-start_time)
y_pred = Nb.predict(X_test)
print('Accuracy_score: ',accuracy_score(y_test,y_pred))

time: 1.688260555267334
Accuracy_score:  0.7818531538104397


In [44]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       agree       0.58      0.14      0.23      1132
    disagree       0.00      0.00      0.00       311
     discuss       0.64      0.47      0.55      2754
   unrelated       0.81      0.95      0.87     10880

    accuracy                           0.78     15077
   macro avg       0.51      0.39      0.41     15077
weighted avg       0.74      0.78      0.75     15077



With Feature Selection

In [45]:
Nb=MultinomialNB()
start_time=time.time()
Nb.fit(X_train_best, y_train)
print("time:",time.time()-start_time)
y_pred = Nb.predict(X_test_best)
print('Accuracy_score: ',accuracy_score(y_test,y_pred))

time: 0.4321284294128418
Accuracy_score:  0.6986137825827419


In [46]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       agree       0.13      0.11      0.12      1132
    disagree       0.21      0.12      0.15       311
     discuss       0.23      0.01      0.03      2754
   unrelated       0.75      0.95      0.84     10880

    accuracy                           0.70     15077
   macro avg       0.33      0.30      0.28     15077
weighted avg       0.60      0.70      0.62     15077



Using PCA

In [47]:
GaussNb=GaussianNB()
start_time=time.time()
GaussNb.fit(pca_train, y_train)
print("time:",time.time()-start_time)
y_pred =GaussNb.predict(pca_test)
print('Accuracy_score: ',accuracy_score(y_test,y_pred))

time: 1.5883519649505615
Accuracy_score:  0.6573588910260661


In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       agree       0.14      0.15      0.15      1132
    disagree       0.26      0.32      0.29       311
     discuss       0.17      0.06      0.08      2754
   unrelated       0.75      0.87      0.81     10880

    accuracy                           0.66     15077
   macro avg       0.33      0.35      0.33     15077
weighted avg       0.59      0.66      0.62     15077



SVM

In [49]:
from sklearn import svm

Withouth Feature Selection

In [ ]:
clf=svm.SVC()
start_time=time.time()
clf.fit(X_train,y_train)
print(time.time()-start_time)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

With Feature selection

In [ ]:
clf=svm.SVC()
start_time=time.time()
clf.fit(X_train_best,y_train)
print(time.time()-start_time)
y_pred = clf.predict(X_test_best)
print(accuracy_score(y_test,y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

With Pca

In [ ]:
clf=svm.SVC()
start_time=time.time()
clf.fit(pca_train,y_train)
print(time.time()-start_time)
y_pred = clf.predict(pca_test)
print(accuracy_score(y_test,y_pred))

In [ ]:
print(classification_report(y_test, y_pred))